In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.decomposition import PCA
from sklearn import svm
import seaborn as sns
from matplotlib.colors import ListedColormap
from sklearn import neighbors
import matplotlib.pyplot as plt
from pandas.plotting import parallel_coordinates
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition
import scipy
from sklearn.manifold import TSNE
from pandas.plotting import parallel_coordinates
from sklearn import decomposition
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.decomposition import PCA
from sklearn import svm
import seaborn as sns
from matplotlib.colors import ListedColormap
from sklearn import neighbors
import matplotlib.pyplot as plt
from pandas.plotting import parallel_coordinates
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition
import scipy
from math import  pi, exp,log, sqrt
from scipy.stats import norm
from sklearn.manifold import TSNE
import seaborn as sn
from sklearn.datasets import make_friedman1
from sklearn.decomposition import SparsePCA
from dask import dataframe as dd

In [14]:
OP_df = pd.read_csv('Option2017_2_clean.csv')
OP_df = OP_df.dropna()
OP_df.head(3)

,Unnamed: 0,currentDate,ExpDate,StrikePrice,Ticker,Type,Last,Implied volatility,Underlaying asset price,Expiration time
0,0,2/17/17,7/21/17,34.799999,TBNK,Call,1.06,0.13250,33.549999,0.421629
1,2,2/17/17,3/17/17,22.500000,ARR,Call,0.08,0.13956,21.709999,0.076660
2,3,2/17/17,3/17/17,20.000000,ARR,Put,0.20,0.32783,21.709999,0.076660


In [3]:
avg = OP_df['Implied volatility'].mean()
avg

0.3108046637052172

In [4]:
def d1(S,K,T,r,sigma):
    
    x = ( log (S/K) + ( ( r + ((sigma**2)/2) )* T  )  ) / ( sigma * ( sqrt ( T ) ) )
    return x 

def d2(S,K,T,r,sigma):
    
    x = ( log (S/K) + ( ( r - ((sigma**2)/2) )* T  )  ) / ( sigma * ( sqrt ( T ) ) )
    return x

In [5]:
def call(S,K,T,r,sigma):

    return S  * norm.cdf( d1(S,K,T,r,sigma) ) - K * (exp(- (r*T) ) ) * norm.cdf( d2(S,K,T,r,sigma) )

def put(S,K,T,r,sigma):

    return   -norm.cdf(- d1(S,K,T,r,sigma)) * S + norm.cdf(-d2(S,K,T,r,sigma)) * K * np.exp(-r * T)

In [6]:

def vega(S, K, T, r, sigma):

    d11 = d1(S,K,T,r,sigma)

    
    vega = S  * np.sqrt(T) * norm.pdf(d11)
    return vega

In [7]:
def newcall(C, S, K,r, T ):
    tol=0.000000000001
    max_iter=100
    sigma = avg

    for i in range(max_iter):
        diff = call(S, K, T, r, sigma) - C

        if abs(diff) < tol:
            break

        sigma = sigma - diff / vega(S, K, T, r, sigma)

    return sigma,i

In [8]:
def newput(C, S, K,r, T ):
    tol=0.000000000001
    max_iter=100
    sigma = avg

    for i in range(max_iter):
        diff = put(S, K, T, r, sigma) - C

        if abs(diff) < tol:
            break

        sigma = sigma - diff / vega(S, K, T, r, sigma)

    return sigma,i

In [9]:
m,it = (newcall( 0.30 , 21.709999 , 22.000000 , 0.03, 0.172485 )) 
print("Implied Volatility",m)
print("Iteration",it)

Implied Volatility 0.10570986506318918
Iteration 3


In [16]:
m

0.10570986506318956

In [15]:
S = np.array( OP_df["Underlaying asset price"]).astype("float")
K = np.array( OP_df["StrikePrice"]).astype("float")
r = .03
T = np.array( OP_df["Expiration time"]).astype("float")
Op = np.array( OP_df["Last"]).astype("float")
Type = np.array( OP_df["Type"]).astype("str")
OP_df.head(3)

,Unnamed: 0,currentDate,ExpDate,StrikePrice,Ticker,Type,Last,Implied volatility,Underlaying asset price,Expiration time
0,0,2/17/17,7/21/17,34.799999,TBNK,Call,1.06,0.13250,33.549999,0.421629
1,2,2/17/17,3/17/17,22.500000,ARR,Call,0.08,0.13956,21.709999,0.076660
2,3,2/17/17,3/17/17,20.000000,ARR,Put,0.20,0.32783,21.709999,0.076660


In [16]:
imp = []
iter = []
for i in range( 0, len(OP_df) ):
    if(Type[i] == "Call"):
        try:
            
            m,it = (newcall( Op[i] , S[i] , K[i] , r , T[i] )) 
            imp.append(m)
            iter.append(it)
        except:
            imp.append(.31)
            iter.append(100)
    else:
        try:
            
            m,it = (newput( Op[i] , S[i] , K[i] , r , T[i] )) 
            imp.append(m)
            iter.append(it)
        except:
            imp.append(.31)
            iter.append(100)

C:\Users\chsha\AppData\Local\Temp/ipykernel_10296/3091123660.py:12: RuntimeWarning: divide by zero encountered in double_scalars
  sigma = sigma - diff / vega(S, K, T, r, sigma)
C:\Users\chsha\AppData\Local\Temp/ipykernel_10296/3075303720.py:3: RuntimeWarning: invalid value encountered in double_scalars
  x = ( log (S/K) + ( ( r + ((sigma**2)/2) )* T  )  ) / ( sigma * ( sqrt ( T ) ) )
C:\Users\chsha\AppData\Local\Temp/ipykernel_10296/3075303720.py:8: RuntimeWarning: invalid value encountered in double_scalars
  x = ( log (S/K) + ( ( r - ((sigma**2)/2) )* T  )  ) / ( sigma * ( sqrt ( T ) ) )


In [17]:
OP_df['imp'] = pd.Series(imp)
OP_df['iter'] = pd.Series(iter)
OP_df.head(3)

,Unnamed: 0,currentDate,ExpDate,StrikePrice,Ticker,Type,Last,Implied volatility,Underlaying asset price,Expiration time,imp,iter
0,0,2/17/17,7/21/17,34.799999,TBNK,Call,1.06,0.13250,33.549999,0.421629,0.162619,3
1,2,2/17/17,3/17/17,22.500000,ARR,Call,0.08,0.13956,21.709999,0.076660,0.132739,5
2,3,2/17/17,3/17/17,20.000000,ARR,Put,0.20,0.32783,21.709999,0.076660,0.341223,4


In [18]:
OP_df["diff"] = OP_df['Implied volatility'] - OP_df['imp']
OP_df['mse'] = OP_df['diff']**2
mse = OP_df['mse'].sum()/len(OP_df)
iteration = OP_df['iter'].sum()/len(OP_df)

In [28]:
mse

0.05391298081554419

In [29]:
iteration

7.363686635944701

In [19]:
n = 1 / ( ( 1 + mse ) * log( 1 + iteration ) )
n
n = 1 / ( ( 1 + mse ) * log( 1 + iteration ) )
print("MSE :",mse)
print("Average Iteration : ",iteration)
print("Average Efficiency",n)

MSE : 0.02987726961515716
Average Iteration :  3.6429493087557603
Average Efficiency 0.6324223254648356


In [20]:
list = []
list.append(mse)
list.append(iteration)
list.append(n)


In [21]:
df = pd.read_csv('result.csv')

In [22]:
df["Newton-Raphson"] = list
df.head(3)

,bisection,Brent,Newton-Raphson,New-Newton,Muller-Bisection,New-Halley,Halley-Irrational
0,0.031976,0.032722,0.029877,0.029913,0.030416,0.025485,0.039493
1,41.053819,16.120337,3.642949,9.617389,26.130753,6.207127,4.181198
2,0.259168,0.340924,0.632422,0.410988,0.294026,0.493729,0.584794


In [23]:
df.to_csv('result.csv', index=False)